In [1]:
from ngboost import NGBRegressor

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
X, y = load_boston(True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

C:\ProgramData\Anaconda3\envs\works\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass return_X_y=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [3]:
ngb = NGBRegressor().fit(X_train, y_train)
y_preds = ngb.predict(X_test)
y_dists = ngb.pred_dist(X_test)

[iter 0] loss=3.6580 val_loss=0.0000 scale=1.0000 norm=6.9011
[iter 100] loss=2.7713 val_loss=0.0000 scale=2.0000 norm=5.2672
[iter 200] loss=2.2075 val_loss=0.0000 scale=2.0000 norm=3.5009
[iter 300] loss=1.9162 val_loss=0.0000 scale=2.0000 norm=2.9835
[iter 400] loss=1.7729 val_loss=0.0000 scale=1.0000 norm=1.3712


In [4]:
test_mse = mean_squared_error(y_preds, y_test)
print('test MSE: ', test_mse)

test MSE:  6.286404645599724


In [6]:
test_nll = -y_dists.logpdf(y_test).mean()
print('test NLL: ', test_nll)

test NLL:  2.7606787244753614
